So far we ran:

* `1 Experimental data prep using rinfino.ipynb`
* `1.1 combine Cell Features.ipynb`

# Chunk up our data

In [1]:
!chunker -h

usage: chunker [-h] --data DATA [--output_file OUTPUT_FILE] --pagesize
               PAGESIZE [--min_chunk_size MIN_CHUNK_SIZE]

optional arguments:
  -h, --help            show this help message and exit
  --data DATA           path to test data (tab delimited genes x samples)
  --output_file OUTPUT_FILE
                        output file prefix (optional)
  --pagesize PAGESIZE   page size (default 10)
  --min_chunk_size MIN_CHUNK_SIZE
                        minimum number of samples in a chunk (default 3),
                        achieved via rebalancing (set to 1 to disable)


In [2]:
!ls ../data/

all_cohorts.cibersort_results.tsv
bladder.tpm.tsv
cohort_newbladder.cibersort.input.classes.datatype_est_counts.txt
provenance.md
rcctils.cellfeatures.tsv
rcctils.celltypes.tsv
singleorigin.cellfeatures.tsv
tcgakirc.tpm.tsv


In [4]:
!ls out

experiment_bladder.test.expression.tsv
experiment_bladder.training.expression.tsv
experiment_bladder.training.xdata.tsv
experiment_rcc.test.chunked.chunk-1-of-2.tsv
experiment_rcc.test.chunked.chunk-2-of-2.tsv
experiment_rcc.test.expression.tsv
experiment_rcc.training.expression.tsv
experiment_rcc.training.xdata.tsv
singleorigin_plus_rcctils.combined.cellfeatures.tsv


In [6]:
!chunker \
--data out/experiment_rcc.test.expression.tsv \
--output_file out/experiment_rcc.test.chunked \
--pagesize 40 --min_chunk_size 10;

Output:
out/experiment_rcc.test.chunked.chunk-1-of-2.tsv  -- 40 samples (X953473f4.9927.4fd4.bfee.8f5908638cc7, ..., a0f62f75.1a15.413e.9960.3cfc09f52b9e)
out/experiment_rcc.test.chunked.chunk-2-of-2.tsv  -- 39 samples (X116f7723.de02.4a81.bec1.cc068019ff38, ..., X0153168a.39ce.47d6.b7f5.ac1e0f312245)


# Compile our Stan models

Create executable version of `modelname.stan`: `make -C $HOME/cmdstan $(pwd)/modelname`. (Note you leave off the `.stan` suffix.)

This creates `modelname` executable. We may need to clear the compiled files out manually to force a rerun of `make`: `rm modelname.hpp modelname`

In [9]:
!ls ../*.stan

../model6.4_negbinom_matrix_correlation_features_oos_optim_otherbucket.stan
../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance.stan


In [12]:
!make -C $HOME/cmdstan $(pwd)/../model6.4_negbinom_matrix_correlation_features_oos_optim_otherbucket

make: Entering directory '/home/jovyan/cmdstan'

--- Translating Stan model to C++ code ---
bin/stanc  /home/jovyan/work/infino-private-2/notebooks/../model6.4_negbinom_matrix_correlation_features_oos_optim_otherbucket.stan --o=/home/jovyan/work/infino-private-2/notebooks/../model6.4_negbinom_matrix_correlation_features_oos_optim_otherbucket.hpp
Model name=model6_4_negbinom_matrix_correlation_features_oos_optim_otherbucket_model
Input file=/home/jovyan/work/infino-private-2/notebooks/../model6.4_negbinom_matrix_correlation_features_oos_optim_otherbucket.stan
Output file=/home/jovyan/work/infino-private-2/notebooks/../model6.4_negbinom_matrix_correlation_features_oos_optim_otherbucket.hpp

--- Linking C++ model ---
g++ -I src -I stan/src -isystem stan/lib/stan_math/ -isystem stan/lib/stan_math/lib/eigen_3.3.3 -isystem stan/lib/stan_math/lib/boost_1.62.0 -isystem stan/lib/stan_math/lib/cvodes_2.9.0/include -Wall -DEIGEN_NO_DEBUG  -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISA

In [13]:
!ls ../model6.4*

../model6.4_negbinom_matrix_correlation_features_oos_optim_otherbucket
../model6.4_negbinom_matrix_correlation_features_oos_optim_otherbucket.hpp
../model6.4_negbinom_matrix_correlation_features_oos_optim_otherbucket.stan


In [14]:
!make -C $HOME/cmdstan $(pwd)/../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance

make: Entering directory '/home/jovyan/cmdstan'

--- Translating Stan model to C++ code ---
bin/stanc  /home/jovyan/work/infino-private-2/notebooks/../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance.stan --o=/home/jovyan/work/infino-private-2/notebooks/../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance.hpp
Model name=model6_5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance_model
Input file=/home/jovyan/work/infino-private-2/notebooks/../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance.stan
Output file=/home/jovyan/work/infino-private-2/notebooks/../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance.hpp

--- Linking C++ model ---
g++ -I src -I stan/src -isystem stan/lib/stan_math/ -isystem stan/lib/stan_math/lib/eigen_3.3.3 -isystem stan/lib/stan_math/lib/boost_1.62.0 -isystem stan/lib/stan_math/lib/cvodes_2.9.0/include -Wall -

In [15]:
!ls ../model6.5*

../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance
../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance.hpp
../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance.stan


In [17]:
!../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance

Usage: ../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance <arg1> <subarg1_1> ... <subarg1_m> ... <arg_n> <subarg_n_1> ... <subarg_n_m>

Begin by selecting amongst the following inference methods and diagnostics,
  sample      Bayesian inference with Markov Chain Monte Carlo
  optimize    Point estimation
  variational  Variational inference
  diagnose    Model diagnostics

Or see help information with
  help        Prints help
  help-all    Prints entire argument tree

Additional configuration available by specifying
  id          Unique process identifier
  data        Input data options
  init        Initialization method: "x" initializes randomly between [-x, x], "0" initializes to 0, anything else identifies a file of values
  random      Random number configuration
  output      File output options

See ../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance <arg1> [ help | help-all ] for details on individual argume

# Execute model

In [18]:
!execute-model -h

/home/jovyan/pyinfino/infino/execute_model.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/opt/conda/bin/execute-model", line 11, in <module>
    load_entry_point('infino', 'console_scripts', 'execute-model')()
  File "/opt/conda/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg/pkg_resources/__init__.py", line 565, in load_entry_point
    return get_distribution(dist).load_entry_point(group, name)
  File "/opt/conda/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg/pkg_resources/__init__.py", line 2598, in load_entry_point
    return ep.load()
  File "/opt/conda/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg/pkg_resources/__init__.py", line 2258, in load
    return self.resol

In [19]:
!ls out

experiment_bladder.test.expression.tsv
experiment_bladder.training.expression.tsv
experiment_bladder.training.xdata.tsv
experiment_rcc.test.chunked.chunk-1-of-2.tsv
experiment_rcc.test.chunked.chunk-2-of-2.tsv
experiment_rcc.test.expression.tsv
experiment_rcc.training.expression.tsv
experiment_rcc.training.xdata.tsv
singleorigin_plus_rcctils.combined.cellfeatures.tsv


In [21]:
!execute-model \
--train_samples out/experiment_rcc.training.expression.tsv \
--train_xdata out/experiment_rcc.training.xdata.tsv \
--train_cellfeatures out/singleorigin_plus_rcctils.combined.cellfeatures.tsv \
--test_samples out/experiment_rcc.test.chunked.chunk-1-of-2.tsv \
--n_chains 4 \
--output_name out/experiment_rcc.test.chunked.chunk-1-of-2 \
--model_executable ../model6.5_negbinom_matrix_correlation_features_oos_optim_otherbucket_samplevariance

/home/jovyan/pyinfino/infino/execute_model.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/opt/conda/bin/execute-model", line 11, in <module>
    load_entry_point('infino', 'console_scripts', 'execute-model')()
  File "/opt/conda/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg/pkg_resources/__init__.py", line 565, in load_entry_point
    return get_distribution(dist).load_entry_point(group, name)
  File "/opt/conda/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg/pkg_resources/__init__.py", line 2598, in load_entry_point
    return ep.load()
  File "/opt/conda/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg/pkg_resources/__init__.py", line 2258, in load
    return self.resol